# Reformat dataset according to BIDS specification for uploading to OpenfMRI

In [6]:
#os and i/o
import os
import numpy as np
import glob
import os.path as op
import pandas as pd
import json

In [7]:
def make_dir(dir_name):
    if not op.exists(dir_name):
        os.mkdir(dir_name)

In [8]:
def make_link(old,new):
    if not op.exists(new):
        cmd = ['ln','-s',old,new]
        cmd = ' '.join(cmd)
        os.system(cmd)

In [9]:
#preliminary housekeeping
home_dir = '/data/home/iballard/mag/'
subj_file = home_dir + 'subjects.txt'
sub_list = list(np.loadtxt(subj_file,'string'))
os.chdir(home_dir)
bids_dir = home_dir + '/mag-BIDS/'
make_dir(bids_dir)

In [10]:
#get new sub names
sub_map = {}
for n,sub in enumerate(sub_list):
    
    if n < 9:
        val = '0' + str(n + 1)
    else:
        val = str(n + 1)

    sub_map[sub] = 'sub-' + val

s = pd.DataFrame({'participant_id': sorted(sub_map.values())})
s.to_csv(bids_dir + 'participants.tsv',index = False)

In [6]:
#initialize directories
for sub in sub_list:
    #make new sub dir
    sub_dir = bids_dir + sub_map[sub]
    make_dir(sub_dir)
    
    #make new anat dir
    anat = op.join(sub_dir, 'anat')
    make_dir(anat)
    
    #make new func dir
    func = op.join(sub_dir, 'func')
    make_dir(func) 
   
    #make new fmap dir
    fmap = op.join(sub_dir, 'fmap')
    make_dir(fmap) 
    

In [7]:
#set up anat files
for sub in sub_list:
    new_anat = op.join(bids_dir, sub_map[sub], 'anat',sub_map[sub] + '_T1w.nii.gz')

    t1 = op.join(home_dir,'data',sub,'anat','st1_0000.nii.gz')
    make_link(t1,new_anat)


In [51]:
#set up func files
for sub in sub_list:
    func = glob.glob(op.join(home_dir,'data',sub,'func','*.nii.gz'))
    
    for f in func:
        run = f.split('/')[-1].split('.')[0][-1]
        new_func = op.join(bids_dir, sub_map[sub], 'func',
                           sub_map[sub] + '_task-mag_run-0' + run + '_bold')
        make_link(f,new_func + '.nii.gz')
        
        #json info
        if sub[0:3]== 'asu':
            info = op.join(home_dir,'asu_subs_info.json')
        else:
            info = op.join(home_dir,'princeton_subs_info.json')
        
        make_link(info,new_func + '.json')


        

In [32]:
#set up onset files
all_data = op.join(home_dir,'subj_params.csv')
all_data = pd.read_csv(all_data)
for sub in sub_list:
    
    if sub == 'asu4':
        subid = 'asu04'
    elif sub == 'asu8':
        subid = 'asu08'
    else:
        subid = sub
        
    sub_df = all_data[all_data['sub'] == subid]
    
    sub_df = sub_df.rename(index=str, columns={"condition": "trial_type"})
    
    sub_df['response_time'] = sub_df['duration']
    sub_df['duration'] = sub_df['duration'] + 5 #fatter HRF in PFC
        
    for run in set(sub_df['run']):

        run_df = sub_df[sub_df['run'] == run]

    
        out_f = op.join(bids_dir, sub_map[sub], 'func',
                        sub_map[sub] + '_task-mag_run-0' + str(run) + '_events.tsv')
                
        run_df.to_csv(out_f, sep = '\t', index = False,
                     columns = ['onset','duration','response_time','amountSS','amountLL',
                               'delaySS','delayLL','choice','SV_SS',
                               'SV_LL','SV_diff'])

In [11]:
#set up fmap files
for sub in sub_list:
    func = glob.glob(op.join(home_dir,'data',sub,'cal','cal*.nii.gz'))
    for f in func:
        run = f.split('/')[-1].split('.')[0][-1]
        new_cal = op.join(bids_dir, sub_map[sub], 'fmap',
                           sub_map[sub] + '_dir-opposing_run-0' + run + '_epi')
        make_link(f,new_cal + '.nii.gz')

        #deal with json file
        json_str  = {"PhaseEncodingDirection": "j-", 
                     "TotalReadoutTime": 0.034,
                     "IntendedFor": "func/" + sub_map[sub] + "_task-mag_run-0" + run + "_bold.nii.gz"}
        
        with open(new_cal + '.json', 'w') as outfile:
            json.dump(json_str, outfile)

In [70]:
#Something had mucked up the HDR files and incorrectly specified the TR as 1 second
for sub in sub_list:
    func = glob.glob(op.join(home_dir,'data',sub,'func','run*.nii.gz'))
    for f in func:
        #copy header to file
        run = f.split('/')[-1].split('.')[0][-1]
        out_hdr = op.join(home_dir,'hdrs',sub + '_' + run +'.txt')
        cmd = ['fslhd', '-x', f, '>', out_hdr]
        cmd = ' '.join(cmd)
        os.system(cmd)
        
        #edit TR
        out_hdr2 = op.join(home_dir,'hdrs',sub + '_' + run +'edit.txt')
        with open(out_hdr, "rt") as fin:
            with open(out_hdr2, "wt") as fout:
                for line in fin:
                    fout.write(line.replace('dt = \'1\'', 'dt = \'2\''))
        
        #write header
        cmd = ['fslcreatehd', out_hdr2, f]
        cmd = ' '.join(cmd)
        os.system(cmd)
